In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [79]:
params_pars_dict = {
    "Thread name": "",
    "Parent name": "",
    "Sampler name": "",
    "Grp Threads": "",
    "Time stamp": "",
    "Response code" : "",
    "Host name" : "",
    "URL" : "",
    "Responce Data" : "",
    "Assertion result":"",
    "Request Data":'',
    "Response Data":"",
    "Response Header":'',
}

columns = [key for key in params_pars_dict]

def xml_jmeter_log_parser(jmeter_xml_log_file_path: str, params_pars_dict) -> list:
    result_array = []

    jmeter_xml_log_file_path = jmeter_xml_log_file_path

    with open(jmeter_xml_log_file_path, encoding='utf-8') as file:
        src = file.read()
        soup = BeautifulSoup(src, "xml")

    testResults_tag = soup.select("testResults")
    # print(len(testResults_tag))

    # Лист из верхних транзакшн контроллеров
    sample = list(testResults_tag[0].children)
    sample_children = [i for i in sample if i!='\n']

    for parent_name in sample_children:

        # перебор TC и поиск TC со статусом fail
        parent = parent_name.find_all('sample', s="false")
        # print(parent[0]['tn'])
        # print(parent[0]['lb'])

        # Thread name (tn)
        # result_list.append(parent[0].get('tn'))
        thread_name_row = parent[0].get('tn').split(" ")
        thread_name = " ".join(thread_name_row[:-1])
        grp_thread = thread_name_row[-1]
        params_pars_dict['Thread name'] = thread_name
        params_pars_dict['Grp Threads'] = grp_thread
        # print(params_pars_dict['Thread name'])
        # Parent name (lb) (TC name)
        # result_list.append(parent[0].get('lb'))
        params_pars_dict['Parent name'] = parent[0].get('lb')
        
        # перебор зафейленных семплеров
        sample = parent[0].find_all(s="false")
        # Sampler name
        # result_list.append(sample[0].get('lb'))
        params_pars_dict['Sampler name'] = sample[0].get('lb')
        # time stamp
        # result_list.append(sample[0].get('ts'))
        # params_pars_dict['Time stamp'] = datetime.fromtimestamp(int(sample[0].get('ts'))/1000).strftime('%d.%m.%Y %H:%M:%S.%f')[:-3]
        params_pars_dict['Time stamp'] = sample[0].get('ts')
        # Response code
        # result_list.append(sample[0].get('rc'))
        params_pars_dict['Response code'] = sample[0].get('rc')
        # Host name
        # result_list.append(sample[0].get('hn'))
        params_pars_dict['Host name'] = sample[0].get('hn')

        # получеие атрибутов запроса/ответа
        # request_response_attr = sample[0].find('java.net.URL')
        # params_pars_dict['Request method, URL'] = " ".join([str(sample[0].find('method').get_text()), str(sample[0].find('java.net.URL').get_text())])
        params_pars_dict['URL'] = sample[0].find('java.net.URL').get_text()
        # print(params_pars_dict['Request method & URL'])
        failure_message_of_assertion_result = sample[0].find('failureMessage')
        p = 'No assertion result' if failure_message_of_assertion_result is None else failure_message_of_assertion_result.get_text().strip()
        params_pars_dict['Assertion result'] = p

        params_pars_dict['Request Data'] = sample[0].find('queryString').get_text().strip()
        params_pars_dict['Responce Data'] = sample[0].find('responseData').get_text().strip()

        params_pars_dict['Response Header'] = sample[0].find('responseHeader').get_text().strip()

        result_array.append([params for key, params in params_pars_dict.items()])

    return result_array

row_data = xml_jmeter_log_parser('C:/GIT_PROJECTS/tools/jmeter_log_parser/data/log2.xml', params_pars_dict=params_pars_dict)

In [80]:

print(columns)
print(row_data[6:7])

['Thread name', 'Parent name', 'Sampler name', 'Grp Threads', 'Time stamp', 'Response code', 'Host name', 'URL', 'Responce Data', 'Assertion result', 'Request Data', 'Response Data', 'Response Header']
[['UC 01.Transfer self', '01.00. - Генерация данных', '01.00.01. - (http) - getDataPoolNextVal (transfer)', '1-3', '1661436004497', '500', 'VDI-P162', 'https://qad.load.homecredit.ru:8081/api/datapool/getitem', '{"statusCode":500,"error":"Internal Server Error","message":"Row or pool is empty!"}', 'No assertion result', '{\r\n\t"pool_name":"L1_0_MSRV_TRANSFER"\r\n}', '', 'HTTP/1.1 500 Internal Server Error\nServer: nginx/1.12.0\nDate: Thu, 25 Aug 2022 14:00:04 GMT\nContent-Type: application/json; charset=utf-8\nContent-Length: 84\nConnection: keep-alive\nvary: Origin\naccess-control-allow-origin: *\naccess-control-expose-headers: xtoken, x-token, fver\nfver: 1.10.50\nsver: 1.0.0\nStrict-Transport-Security: max-age=15724800; includeSubDomains']]


In [77]:

def row_data_to_data_frame(row_data, columns):
    df = pd.DataFrame(row_data, columns=columns)
    df = df.sort_values(by=['Thread name', 'Parent name', 'Sampler name', 'Time stamp'])
    # pivot_df = pd.pivot_table(df, index=['Time stamp'])
    return df

df = row_data_to_data_frame(row_data=row_data, columns=columns)
df[0:6]

ValueError: 13 columns passed, passed data had 12 columns

In [7]:
print(df['Thread name'].unique())

['UC 01.Transfer self' 'UC 02.Transfer card'
 'UC 04.Transfer and type self']


In [8]:
df[df['Thread name'].isin(['UC 01.Transfer self'])]['Parent name'].unique()

array(['01.00. - Генерация данных', '01.02. - Создание сессии (self)',
       '01.08. - Проверка СМС и исполнение платежа'], dtype=object)

In [9]:
df[df['Parent name'].isin(['01.00. - Генерация данных'])]['Sampler name'].unique()

array(['01.00.01. - (http) - getDataPoolNextVal (transfer)'], dtype=object)

In [10]:
df[df['Sampler name'].isin(['01.00.01. - (http) - getDataPoolNextVal (transfer)'])]['Time stamp'].unique()

array(['25.08.2022 18:00:03.773', '25.08.2022 18:00:04.497',
       '25.08.2022 18:14:09.681', '25.08.2022 18:20:47.507',
       '25.08.2022 18:27:25.798'], dtype=object)

In [16]:


for thread_name in df["Thread name"].unique():
    print(thread_name)
    for parent_name in df[df['Thread name'].isin([thread_name])]['Parent name'].unique():
        print(parent_name)
        for sampler_name in df[df["Parent name"].isin([parent_name])]["Sampler name"].unique():
            print(f"Sampler name {sampler_name}")
            for time_stamp in df[df["Sampler name"].isin([sampler_name])]["Time stamp"].unique():
                res_code = df[df["Sampler name"].isin([sampler_name])]["Response code"].unique()[0]
                print(time_stamp, res_code)


UC 01.Transfer self
01.00. - Генерация данных
Sampler name 01.00.01. - (http) - getDataPoolNextVal (transfer)
25.08.2022 18:00:03.773 500
25.08.2022 18:00:04.497 500
25.08.2022 18:14:09.681 500
25.08.2022 18:20:47.507 500
25.08.2022 18:27:25.798 500
01.02. - Создание сессии (self)
Sampler name 01.02.01 - /transfer/v1/self/transfer
25.08.2022 18:24:41.353 500
25.08.2022 18:26:02.100 500
01.08. - Проверка СМС и исполнение платежа
Sampler name 01.08.01. - /transfer/v1/self/transfer/huid/check
25.08.2022 17:49:55.364 500
25.08.2022 18:09:27.991 500
25.08.2022 18:15:24.308 500
25.08.2022 18:22:02.343 500
25.08.2022 18:22:44.724 500
UC 02.Transfer card
02.00. - Генерация данных
Sampler name 02.00.01. - (http) - getDataPoolNextVal (transfer)
25.08.2022 18:07:31.697 500
25.08.2022 18:14:09.771 500
02.05. - Подтверждение платежа
Sampler name 02.05.01. - /transfer/v1/card/transfer/huid/confirm
25.08.2022 18:08:28.989 400
25.08.2022 18:12:59.145 400
25.08.2022 18:15:07.382 400
02.07. - Проверка С

In [21]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('static/'))
template = env.get_template('accordion_thread_list.html')
print(template.render(name=u'Петя'))

TemplateSyntaxError: Encountered unknown tag 'extend'.